# 1. 回顾TensorBoard

## 1.1 启动
登陆虚拟机，运行以下命令启动TensorBoard:
```
docker exec -it datalab /bin/sh
tensorboard -logdir /content  (将/content替换成具体日志的路径)
```

## 1.2 节点名称(node name)和命名空间(name scope)
为了让计算图显示的更加有序，在编写TensorFlow代码时应该正确的使用。

计算图中的每个节点都需要有名称，可以使用`name`参数进行定义：
```
Weights = tf.Variable(..., name='W')
```

需要将每个节点放置在合适的命名空间内：

```
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.reduce_sum(...))
```

## 1.3 计算图

通过`tf.Graph()`创建新的计算图，并将相应的节点都放入该计算图中：
```
with tf.Graph().as_default() as g:
    with tf.name_scope('inputs'):
    ...
```

## 1.4 TensorBoard参考
1. TensorFlow 2017 Dev Summit (TensorBoard): https://github.com/mamcgrath/TensorBoard-TF-Dev-Summit-Tutorial/blob/master/slides.pdf
2. TensorBoard官网参考资料： https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard?hl=zh-cn

# 2. 重新开始
以下是上次的作业，进行重构后的FFNN代码：

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.zeros([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]), name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    return outputs

with tf.Graph().as_default() as g:
    with tf.name_scope('input'):
        # X: 输入
        X = tf.placeholder(tf.float32, [None, 784], name="X")
        # Y_: 标签
        Y_ = tf.placeholder(tf.float32, [None, 10])

    # L1：200个神经元
    Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

    # L2：100个神经元
    Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

    # L3: 60个神经元
    Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

    # L4: 30个神经元
    Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

    # L5: 10个神经元
    Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

    # Output
    Y = tf.nn.softmax(Ylogits)

    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
        

    # 优化算法
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        for i in range(1000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            _ = sess.run([train_step], feed_dict={X: batch_xs, Y_: batch_ys})

            if i%100 == 0:
                print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

/Users/lyon/.pyenv/versions/3.6.4/envs/scikit-learn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.0982
0.1135
0.1135
0.1135
0.1135
0.1135
0.101
0.1135
0.1135
0.1135


运行上述代码后，发现准确率一直在10%，我们需要更多的信息去诊断问题！

## 2.1 Scalar数据
TensorBoard 通过读取 TensorFlow 的事件文件来运行。TensorFlow 的事件文件包含运行 TensorFlow 时生成的总结数据。下面是 TensorBoard 中总结数据的一般生命周期。

1. 创建您想从中收集总结数据的 TensorFlow 图，然后再选择您想在哪个节点标注总结指令(summary)。假设您正在训练一个神经网络，用于识别 MNIST 数据。您可能希望记录随着时间的推移，学习速度如何变化，以及目标函数如何变化。通过向节点附加 tf.summary.scalar op 来分别输出学习速度和误差。例如：
```
    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
        
    # 输出损失函数的值(类型：标量)
    tf.summary.scalar('cross_entropy', cross_entropy)
    
    ...
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    # 输出损失函数的值(类型：标量)
    tf.summary.scalar('accuracy', accuracy)
```
有关所有可用的总结指令的详细信息，可查看：https://www.tensorflow.org/api_guides/python/summary?hl=zh-cn

2. 在 TensorFlow 中，只有当您运行指令时，指令才会执行，或者另一个 op 依赖于指令的输出时，指令才会运行。我们刚才创建的这些总结节点都围绕着您的图：您目前运行的 op 都不依赖于这些节点的结果。因此，为了生成总结信息，我们需要运行所有这些总结节点。这样的手动操作是枯燥而乏味的，因此可以使用 tf.summary.merge_all 来将这些操作合并为一个 op，从而生成所有总结数据。
```
    # 实际使用中会输出多个信息，惯用做法是将所有输出打包
    merged = tf.summary.merge_all()
```

3. 您可以执行该合并的总结 op，它会在特定步骤将所有总结数据生成一个序列化的 Summary protobuf 对象。最后，如要将此总结数据写入磁盘，请将此总结 protobuf 对象传递给 tf.summary.FileWriter。FileWriter 的构造函数中包含了参数 logdir，logdir 参数非常重要，所有事件都会写到它所指的目录下。**注意：输出运行时的概要信息(summary)是在会话运行的时候进行的**
```
with tf.Session() as sess:
    ...
    summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})

    for i in range(1000):
        ...
        # 将每次迭代过程中的信息，通过FileWriter输出
        writer.add_summary(summary, i)
        ...
    ...
```

练习：将上述添加概要信息的代码，添加到下面代码块合适的位置，运行TensorBoard查看输出信息。

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

with tf.Graph().as_default() as g:
    with tf.name_scope('input'):
        # X: 输入
        X = tf.placeholder(tf.float32, [None, 784], name="X")
        # Y_: 标签
        Y_ = tf.placeholder(tf.float32, [None, 10])

    # L1：200个神经元
    Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

    # L2：100个神经元
    Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

    # L3: 60个神经元
    Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

    # L4: 30个神经元
    Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

    # L5: 10个神经元
    Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

    # Output
    Y = tf.nn.softmax(Ylogits)

    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
    
    tf.summary.scalar('cross_entropy', cross_entropy)
    
    # 优化算法
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        for i in range(1000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})
            
            writer.add_summary(summary, i)
            
            if i%100 == 0:
                print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.0958
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135


## 2.2 模型参数

经过2.1的努力，我们看到了迭代过程中损失函数和准确率的变化，如下图所示：
![损失函数和准确率](http://p811pjpxl.bkt.clouddn.com/15-1.png)

从上述图表中我们发现，损失函数的值在训练过程中并没有明显下降。我们需要可视化更多的数据，下一步我们要将W(Weights)和B(Biases)进行可视化，使用`tf.summary.histogram`。

```
    tf.summary.histogram('weights', Weights)
    tf.summary.histogram('biases', biases)
```

API参考：https://www.tensorflow.org/api_docs/python/tf/summary/histogram?hl=zh-cn

**练习**：
```
    tf.summary.histogram('weights', Weights)
    tf.summary.histogram('biases', biases)
```
将上述代码添加到下列代码块合适的地方，输出神经网络中的参数Weights和biases。通过TensorBoard查看相应的信息。

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.zeros([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]), name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        tf.summary.histogram('weights', Weights)
        tf.summary.histogram('biases', biases)
        
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    return outputs

with tf.Graph().as_default() as g:
    with tf.name_scope('input'):
        # X: 输入
        X = tf.placeholder(tf.float32, [None, 784], name="X")
        # Y_: 标签
        Y_ = tf.placeholder(tf.float32, [None, 10])

    # L1：200个神经元
    Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

    # L2：100个神经元
    Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

    # L3: 60个神经元
    Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

    # L4: 30个神经元
    Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

    # L5: 10个神经元
    Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

    # Output
    Y = tf.nn.softmax(Ylogits)

    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
        
    tf.summary.scalar('cross_entropy', cross_entropy)
        
    # 优化算法
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        for i in range(1000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})

            writer.add_summary(summary, i)
            
            if i%100 == 0:
                print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.1135
0.1135
0.1135
0.1028
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135


## 2.3 初始化参数

通过可视化Weights和biases的值，我们发现他们都在0徘徊，好像'根本没有学习到任何有用的经验'! 问题主要出在，我们的代码将他们都初始化为0！！！

TensorFlow提供了如下方法对于变量进行初始化：
![初始化方法](http://p811pjpxl.bkt.clouddn.com/15-2.png)

我们需要选用上图中合适的方法对网络模型中的Weights和biases参数进行初始化。
例如将Weights初始化为随机值(符合正态分布)：
```
Weights = tf.Variable(tf.truncated_normal([in_size, out_size]), name='W')
```
例如将biases初始化为0.1:
```
biases = tf.Variable(tf.constant(0.1, shape=[1, out_size]), name='b')
```

参考：
- `tf.constant`:
https://www.tensorflow.org/api_docs/python/tf/constant?hl=zh-cn

- `tf.truncated_normal:`
https://www.tensorflow.org/versions/master/api_docs/python/tf/truncated_normal?hl=zh-cn

练习：修改函数`add_layer`中变量的初始化代码，并运行。通过TensorBoard查看损失函数`cross_entropy`的变化。

In [21]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.zeros([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros(shape=[1, out_size]), name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    return outputs

with tf.Graph().as_default() as g:
    with tf.name_scope('input'):
        # X: 输入
        X = tf.placeholder(tf.float32, [None, 784], name="X")
        # Y_: 标签
        Y_ = tf.placeholder(tf.float32, [None, 10])

    # L1：200个神经元
    Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

    # L2：100个神经元
    Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

    # L3: 60个神经元
    Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

    # L4: 30个神经元
    Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

    # L5: 10个神经元
    Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

    # Output
    Y = tf.nn.softmax(Ylogits)

    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
        
    tf.summary.scalar('cross_entropy', cross_entropy)
        
    # 优化算法
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        for i in range(1000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})

            writer.add_summary(summary, i)
            
            if i%100 == 0:
                print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.098
0.1135
0.1135
0.1135
0.1135
0.1135
0.1028
0.1135
0.1028
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135


## 2.4 迭代次数

查看运行2.3程序后的损失函数图，发现损失函数呈下降趋势，但是并未收敛，说明训练并未完成，需要增加训练次数。
![image.png](http://p811pjpxl.bkt.clouddn.com/15-3.png)

**练习**：修改训练迭代次数，直到通过TensorBoard查看到损失函数值收敛。

In [19]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 导入数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.truncated_normal([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.1, shape=[1, out_size]), name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    return outputs

with tf.Graph().as_default() as g:
    with tf.name_scope('input'):
        # X: 输入
        X = tf.placeholder(tf.float32, [None, 784], name="X")
        # Y_: 标签
        Y_ = tf.placeholder(tf.float32, [None, 10])

    # L1：200个神经元
    Y1 = add_layer(X, 784, 200, tf.nn.sigmoid)

    # L2：100个神经元
    Y2 = add_layer(Y1, 200, 100, tf.nn.sigmoid)

    # L3: 60个神经元
    Y3 = add_layer(Y2, 100, 60, tf.nn.sigmoid)

    # L4: 30个神经元
    Y4 = add_layer(Y3, 60, 30, tf.nn.sigmoid)

    # L5: 10个神经元
    Ylogits = add_layer(Y4, 30, 10, tf.nn.sigmoid)

    # Output
    Y = tf.nn.softmax(Ylogits)

    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
        
    tf.summary.scalar('cross_entropy', cross_entropy)
        
    # 优化算法
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
        
    # 计算准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        for i in range(10000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            summary, _ = sess.run([merged, train_step], feed_dict={X: batch_xs, Y_: batch_ys})

            writer.add_summary(summary, i)
            
            if i%100 == 0:
                print(accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.1008
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1009
0.1015
0.1019
0.1021
0.1021
0.1026
0.1033
0.1042
0.1142
0.1239
0.1335
0.1401
0.1439
0.1612
0.1873
0.1966
0.2531
0.2798
0.3467
0.4451
0.5809
0.7575
0.83
0.8441
0.8478
0.8512
0.8532
0.8557
0.859
0.8619
0.8619
0.8658
0.867
0.8685
0.8716
0.8726
0.8734
0.8741
0.877
0.8782
0.8804
0.8817
0.8821
0.8842
0.8838
0.8862
0.8878
0.8885
0.8887
0.8893
0.8907
0.8918
0.8916
0.8934
0.894
0.8948
0.8961
0.8957
0.8983
0.9004
0.8976
0.9003
0.9007
0.8999
0.9011
0.9026
0.9034
0.9026
0.9019
0.9038
0.9044
0.9053
0.9054
0.9054
0.9062
0.9068
0.9072
0.9071
0.9082
0.9085
0.9083
0.9089
0.9094


可以看到通过增加训练迭代次数，训练效果得到明显提升：
![image.png](http://p811pjpxl.bkt.clouddn.com/15-4.png)

# 参考
1. https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/4-1-tensorboard1/

# 参考答案

## 1.3

In [2]:
import tensorflow as tf

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    return outputs


with tf.name_scope('inputs'):
    # define placeholder for inputs to network
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)

# add output layer
prediction = add_layer(l1, 10, 1, activation_function=None)

# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

with tf.Session() as sess:
    # 将当前会话中的计算图保存
    writer = tf.summary.FileWriter("logs/", sess.graph)

    init = tf.global_variables_initializer()
    sess.run(init)

## 1.4

In [2]:
import tensorflow as tf

def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    return outputs


with tf.Graph().as_default() as g:
    with tf.name_scope('inputs'):
        # define placeholder for inputs to network
        xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
        ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

    # add hidden layer
    l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)

    # add output layer
    prediction = add_layer(l1, 10, 1, activation_function=None)

    # the error between prediciton and real data
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                            reduction_indices=[1]))
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    with tf.Session() as sess:
        # 将当前会话中的计算图保存
        writer = tf.summary.FileWriter("logs/", sess.graph)

        init = tf.global_variables_initializer()
        sess.run(init)